In [1]:
! unzip /content/drive/MyDrive/Oranges/train.zip

Archive:  /content/drive/MyDrive/Oranges/train.zip
   creating: train/
   creating: train/Blood Oranges_transform/
  inflating: train/Blood Oranges_transform/1.png  
  inflating: train/Blood Oranges_transform/10.png  
  inflating: train/Blood Oranges_transform/11.png  
  inflating: train/Blood Oranges_transform/12.png  
  inflating: train/Blood Oranges_transform/13.png  
  inflating: train/Blood Oranges_transform/14.png  
  inflating: train/Blood Oranges_transform/15.png  
  inflating: train/Blood Oranges_transform/16.png  
  inflating: train/Blood Oranges_transform/17.png  
  inflating: train/Blood Oranges_transform/18.png  
  inflating: train/Blood Oranges_transform/19.png  
  inflating: train/Blood Oranges_transform/2.png  
  inflating: train/Blood Oranges_transform/20.png  
  inflating: train/Blood Oranges_transform/3.png  
  inflating: train/Blood Oranges_transform/4.png  
  inflating: train/Blood Oranges_transform/5.png  
  inflating: train/Blood Oranges_transform/6.png  
  infla

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import time
import copy
import numpy as np
from tqdm import tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
# Define the augmentations and transformations
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(20),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.RandomErasing(),  # Random erasing for regularization
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load the dataset
full_dataset = datasets.ImageFolder(root='/content/train', transform=transform_train)

# Split into train and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Apply transforms
train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_val

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [25]:
from torchvision.models import ResNet50_Weights, MobileNet_V2_Weights, Inception_V3_Weights, EfficientNet_V2_S_Weights

def get_model(model_name, num_classes, pretrained=True):
    if model_name == 'resnet':
        model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'mobilenet':
        model = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1 if pretrained else None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == 'inception':
        model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1 if pretrained else None, aux_logits=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'efficientnet':
        model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1 if pretrained else None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Model not supported")

    return model


In [20]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    dataloaders = {'train': train_loader, 'val': val_loader}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if validation accuracy is improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [23]:
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')
    return accuracy


In [8]:
def save_model(model, model_name):
    torch.save(model.state_dict(), f'{model_name}_best.pth')
    print(f'Model saved as {model_name}_best.pth')


In [57]:
# Assuming dataloaders is a dictionary with 'train', 'val', and 'test' dataloaders
num_classes = 5  # Change this according to your dataset
models_to_train = ['resnet', 'mobilenet', 'efficientnet']
accuracies = {}

for model_name in models_to_train:
    print(f'\nTraining {model_name}...\n')

    model = get_model(model_name, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    model = train_model(model, train_loader,val_loader, criterion, optimizer, num_epochs=100)
    accuracy = evaluate_model(model,val_loader)

    accuracies[model_name] = accuracy

    save_model(model, model_name)



Training resnet...

Epoch 0/99
----------


100%|██████████| 9/9 [00:01<00:00,  8.51it/s]


train Loss: 1.6501 Acc: 0.2113


100%|██████████| 3/3 [00:00<00:00, 23.93it/s]


val Loss: 1.6202 Acc: 0.1111

Epoch 1/99
----------


...

val Loss: 0.7573 Acc: 0.7778

Epoch 99/99
----------


100%|██████████| 9/9 [00:01<00:00,  6.20it/s]


train Loss: 0.0144 Acc: 1.0000


100%|██████████| 3/3 [00:00<00:00, 12.78it/s]


val Loss: 0.7503 Acc: 0.7778

Best val Acc: 0.8333
Accuracy on test set: 83.33%
Model saved as efficientnet_best.pth


In [65]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import json

def load_model(model_name, num_classes, model_path):
    model = get_model(model_name, num_classes, pretrained=False)  # Initialize model architecture
    model.load_state_dict(torch.load(model_path))  # Load the saved weights
    model.eval()  # Set the model to evaluation mode
    return model

# Example: Load the Inception model
model_name = 'efficientnet'
num_classes = 5  # Adjust according to your dataset
model_path = '/content/efficientnet_best.pth'  # Path to the saved model
model = load_model(model_name, num_classes, model_path)
model = model.to(device)


In [69]:
def preprocess_image(image_path, model_name):
    if model_name == 'inception':
        transform = transforms.Compose([
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def predict(image_path, model, model_name, class_names):
    # Preprocess the image
    image = preprocess_image(image_path, model_name)
    image = image.to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(image)
        print("Model output:", outputs)
        probabilities = torch.softmax(outputs, dim=1)
        print("Class probabilities:", probabilities)
        _, predicted = torch.max(outputs, 1)

    # Map the prediction to the class label
    print(predicted.item())
    predicted_class = class_names[predicted.item()]
    return predicted_class

# Example usage
image_path = '/content/train/Tangelo_transform/1.png'  # Path to the image you want to classify
class_names = ['Blood Oranges_transform', 'Navel_transform', 'Tangelo_transform', 'Tangerine_transform', 'cara cara_transform']  # Replace with your actual class names

predicted_class = predict(image_path, model, model_name, class_names)
print(f'Predicted class: {predicted_class}')


Model output: tensor([[-0.1472, -1.5155,  4.6386, -1.0563, -2.9023]], device='cuda:0')
Class probabilities: tensor([[8.2290e-03, 2.0947e-03, 9.8584e-01, 3.3155e-03, 5.2338e-04]],
       device='cuda:0')
2
Predicted class: Tangelo_transform


In [70]:
! cp "/content/efficientnet_best.pth" "/content/drive/MyDrive/Oranges/FINE TUNED MODELS"

In [71]:
! cp "/content/mobilenet_best.pth" "/content/drive/MyDrive/Oranges/FINE TUNED MODELS"

In [72]:
! cp "/content/resnet_best.pth" "/content/drive/MyDrive/Oranges/FINE TUNED MODELS"